# 中文评论情感分析 -- 正面评论和负面评论的分类

本次作业，我基于中文分词和TF-IDF算法，利用jieba库和机器学习sklearn以及keras库，实现了一个中文评论情感分析的模型，该模型能够以超过90%的准确率判断出一段评论是积极的还是消极的。

本次作业的数据集来自网上提取的苹果店铺评论，停用词表采用哈工大中文停用词表。

## 0 导入相关库

In [19]:
import jieba
import pandas as pd
import csv
import time
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection 
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.optimizers import Adam
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

## 1 数据清洗

### 1.1 原始数据做分词及剔除停用词

In [20]:
# 创建停用词列表
def stopwordslist():
    stopwords = [line.strip() for line in open(r'E:\Data\ChineseData\textemotion/HGD_StopWords.txt',encoding='UTF-8').readlines()]
    return stopwords

In [21]:
# 对句子进行中文分词
def seg_depart(sentence):
    # 对文档中的每一行进行中文分词
    sentence_depart = jieba.cut(sentence.strip())
    # 引进停用词列表
    stopwords = stopwordslist()
    # 输出结果为outstr
    outstr = ''
    # 去停用词
    for word in sentence_depart:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

In [22]:
filename = r'E:\Data\ChineseData\textemotion/ALL_Comment.txt' # 原始数据
outfilename = r'E:\Data\ChineseData\textemotion/stop_seg_word.txt' # 经过停用词表筛选之后的数据
inputs = open(filename, 'r', encoding='UTF-8')
outputs=open(outfilename, 'w', encoding='UTF-8')
# 将输出结果写入out中
count=0
for line in inputs:
    line_seg = seg_depart(line)
    outputs.writelines(line_seg + '\n')
    count=count+1
print("一共处理了",count,"条数据")
outputs.close()
inputs.close()
print("删除停用词和分词成功！！！")

一共处理了 7848 条数据
删除停用词和分词成功！！！


### 1.2 将数据和标签对应，存进csv文件

In [23]:
data = pd.DataFrame()
#将txt文件中的数据按行写入csv文件
with open(r'E:\Data\ChineseData\textemotion/stop_seg_word.txt', encoding='utf-8') as f:
    line = f.readlines()
    line = [i.strip() for i in line]
    print(len(line))
#建立评论这一列，将数据写入
data['评论'] = line
data

7848


,评论
0,﻿ 真机 很漂亮 体验 看 评论 说 发烫 情况 几天 摄像头 有点 现在 挺 待机 感觉 ...
1,外观 好看 黄色 特别 喜欢 裸机 非常 好看 旁边 黑边 略微 习惯 还好 面部 解锁 非...
2,手感 光滑 圆润 质感 不像 中 略厚 一点点 适合 女孩子 无碍 诟病 屏幕 黑边 我觉 ...
3,这次 iphone11 值得 购买 拍摄 方面 很大 提升 夜景 模式 测试 看 原图 噪点...
4,手机 不错 毕竟 系统 强大 拍照 效果 音效 都 提升 操作 顺滑 待机 期间 长刚 激活...
...,...
7843,京东 老 客服 这次 体验 非常 愉快 标签 损坏 重贴 痕迹 重度 怀疑 退换货 翻新 机...
7844,买回来 没 仔细 看 带上 买回来 软套 前 几天 打开 一看 背面 有个 厘米 口子 非常...
7845,差评 差评 一点 都 不好 怀疑 翻新 拆封 机 左边 缝隙 右边 无比 贴合 客服 还 说...
7846,真是 服 京东 全 假货 真信 买 手里 激活 超过 天 白屏 开机 充电 京东...


In [24]:
with open(r'E:\Data\ChineseData\textemotion/All_label.txt', "r",encoding='utf-8') as f:
    all_label=f.readlines()
    print(type(all_label))
    print(len(all_label))

all_labels=[]
for element in all_label:
    all_labels.extend(element.split(','))

#建立“评分”这一列，将标签写入
data['评分'] = all_labels
data.to_csv(r'E:\Data\ChineseData\textemotion/reviews_score_update.csv')
data

<class 'list'>
1


,评论,评分
0,﻿ 真机 很漂亮 体验 看 评论 说 发烫 情况 几天 摄像头 有点 现在 挺 待机 感觉 ...,1
1,外观 好看 黄色 特别 喜欢 裸机 非常 好看 旁边 黑边 略微 习惯 还好 面部 解锁 非...,1
2,手感 光滑 圆润 质感 不像 中 略厚 一点点 适合 女孩子 无碍 诟病 屏幕 黑边 我觉 ...,1
3,这次 iphone11 值得 购买 拍摄 方面 很大 提升 夜景 模式 测试 看 原图 噪点...,1
4,手机 不错 毕竟 系统 强大 拍照 效果 音效 都 提升 操作 顺滑 待机 期间 长刚 激活...,1
...,...,...
7843,京东 老 客服 这次 体验 非常 愉快 标签 损坏 重贴 痕迹 重度 怀疑 退换货 翻新 机...,0
7844,买回来 没 仔细 看 带上 买回来 软套 前 几天 打开 一看 背面 有个 厘米 口子 非常...,0
7845,差评 差评 一点 都 不好 怀疑 翻新 拆封 机 左边 缝隙 右边 无比 贴合 客服 还 说...,0
7846,真是 服 京东 全 假货 真信 买 手里 激活 超过 天 白屏 开机 充电 京东...,0


## 2 数据集划分

In [25]:
#将数据进行读取
data=pd.read_csv(r'E:\Data\ChineseData\textemotion/reviews_score_update.csv',index_col=0)
data.head()

,评论,评分
0,﻿ 真机 很漂亮 体验 看 评论 说 发烫 情况 几天 摄像头 有点 现在 挺 待机 感觉 ...,1
1,外观 好看 黄色 特别 喜欢 裸机 非常 好看 旁边 黑边 略微 习惯 还好 面部 解锁 非...,1
2,手感 光滑 圆润 质感 不像 中 略厚 一点点 适合 女孩子 无碍 诟病 屏幕 黑边 我觉 ...,1
3,这次 iphone11 值得 购买 拍摄 方面 很大 提升 夜景 模式 测试 看 原图 噪点...,1
4,手机 不错 毕竟 系统 强大 拍照 效果 音效 都 提升 操作 顺滑 待机 期间 长刚 激活...,1


In [26]:
#划分数据集为训练集和预测集
train_x,test_x,train_y,test_y=model_selection.train_test_split(data.评论.values.astype('U'),data.评分.values,test_size=0.1,random_state=49)
 
#划分完毕，查看数据形状
print(train_x.shape,test_x.shape)

(7063,) (785,)


## 3 特征提取 -- TF-IDF

In [27]:
#根据哈工大中文停用词表进行去停用词
def get_stopwords(stop_word_file):
    with open(stop_word_file, 'r', encoding='utf-8') as f:
        stopwords=f.read()
    stopwords_list=stopwords.split('\n')
    custom_stopwords_list=[i for i in stopwords_list]
    return custom_stopwords_list

In [28]:
#获得由停用词组成的列表
stop_words_file = r'E:\Data\ChineseData\textemotion/HGD_StopWords.txt'
stopwords = get_stopwords(stop_words_file)

In [29]:
'''
使用TfidfVectorizer()对数据进行特征的提取，投放到不同的模型中进行实验
'''
#使用TF-IDF进行特征的提取，对分词后的中文语句做向量化。
TF_Vec=TfidfVectorizer(max_df=0.8,
                       min_df = 3,
                       stop_words=frozenset(stopwords)
                      )
#拟合数据，将数据准转为标准形式，一般使用在训练集中
train_x_tfvec=TF_Vec.fit_transform(train_x)
#通过中心化和缩放实现标准化，一般使用在测试集中
test_x_tfvec=TF_Vec.transform(test_x)
train_x_tfvec.shape

C:\Users\21548\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', 'max', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(7063, 3597)

## 4 搭建神经网络，进行数据训练和预测

In [30]:
np.random.seed(0)
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(train_x_tfvec.shape[1],)))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(32, activation='relu'))

model.add(layers.Dense(2, activation='softmax'))

In [31]:
model.compile(optimizer=Adam(0.0005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
history = model.fit(train_x_tfvec,
                    train_y,
                    epochs=10,
                    batch_size=32)

Epoch 1/10
7063/7063 [==============================] - 1s 201us/step - loss: 0.3026 - accuracy: 0.8569
Epoch 2/10
7063/7063 [==============================] - 1s 187us/step - loss: 0.0727 - accuracy: 0.9759
Epoch 3/10
7063/7063 [==============================] - 1s 185us/step - loss: 0.0367 - accuracy: 0.9897
Epoch 4/10
7063/7063 [==============================] - 1s 191us/step - loss: 0.0212 - accuracy: 0.9943
Epoch 5/10
7063/7063 [==============================] - 1s 188us/step - loss: 0.0128 - accuracy: 0.9970
Epoch 6/10
7063/7063 [==============================] - 2s 213us/step - loss: 0.0087 - accuracy: 0.9982
Epoch 7/10
7063/7063 [==============================] - 1s 204us/step - loss: 0.0061 - accuracy: 0.9984
Epoch 8/10
7063/7063 [==============================] - 1s 172us/step - loss: 0.0045 - accuracy: 0.9990
Epoch 9/10
7063/7063 [==============================] - 1s 172us/step - loss: 0.0038 - accuracy: 0.9993
Epoch 10/10
7063/7063 [==============================] - 1s 172u

In [33]:
test_loss, test_acc = model.evaluate(test_x_tfvec,test_y)
print('test_acc: ',test_acc)

785/785 [==============================] - 0s 65us/step
test_acc:  0.9363057613372803


## 5 使用逻辑回归进行分类

In [34]:
'''
使用TF_IDF提取的向量当作数据特征传入模型
'''
#构建模型之前首先将包进行导入
start_time=time.time()
#创建模型
lr = linear_model.LogisticRegression(penalty='l2', C=1, solver='liblinear', max_iter=1000, multi_class='ovr')
model = GridSearchCV(lr, cv=3, param_grid={
        'C': np.logspace(0, 4, 30),
        'penalty': ['l1', 'l2']
    })
#模型拟合tf-idf拿到的数据
model.fit(train_x_tfvec,train_y)
#查看模型自己拟合的最优参数
print('最优参数：', model.best_params_)
#在训练时查看训练集的准确率
pre_train_y=model.predict(train_x_tfvec)
#在训练集上的正确率
train_accracy=accuracy_score(pre_train_y,train_y)
#训练结束查看预测 输入验证集查看预测
pre_test_y=model.predict(test_x_tfvec)
#查看在测试集上的准确率
test_accracy = accuracy_score(pre_test_y,test_y)
print('使用TF-IDF提取特征使用逻辑回归,让模型自适应参数，进行模型优化\n训练集:{0}\n测试集:{1}'.format(train_accracy,test_accracy))
end_time=time.time()
print("使用模型优化的程序运行时间为",end_time-start_time)

最优参数： {'C': 6.7233575364993365, 'penalty': 'l2'}
使用TF-IDF提取特征使用逻辑回归,让模型自适应参数，进行模型优化
训练集:0.9881070366699702
测试集:0.9579617834394905
使用模型优化的程序运行时间为 4.295543909072876


## 6 使用随机森林分类进行分类

In [35]:
### Random Forest Classifier 随机森林分类器 
from sklearn.ensemble import RandomForestClassifier 
import time
start_time=time.time()
#创建模型
Rfc = RandomForestClassifier(n_estimators=8)
#拟合从CounterfVectorizer拿到的数据
Rfc.fit(train_x_tfvec,train_y)
#在训练时查看训练集的准确率
pre_train_y=Rfc.predict(train_x_tfvec)
#在训练集上的正确率
train_accracy=accuracy_score(pre_train_y,train_y)
#训练结束查看预测 输入测试集查看预测
pre_test_y=Rfc.predict(test_x_tfvec)
#查看在测试集上的准确率
test_accracy = accuracy_score(pre_test_y,test_y)
print('使用CounterfVectorizer提取特征使用随机森林分类器的准确率\n训练集:{0}\n测试集:{1}'.format(train_accracy,test_accracy))
end_time=time.time()
print("使用随机森林分类器的程序运行时间为",end_time-start_time)

使用CounterfVectorizer提取特征使用随机森林分类器的准确率
训练集:0.9954693473028459
测试集:0.9286624203821656
使用随机森林分类器的程序运行时间为 0.20747137069702148


## 总结

本次作业实现了一个准确率较高而分类器，能够判断出一条评论的情感倾向（是正面评价还是负面评价），其中用到了：
* jieba库进行中文分词
* TF-IDF库进行数据的特征提取
* sklearn库进行数据集的划分以及逻辑回归和随机森林的分类
* keras库进行神经网络的搭建、训练以及预测

对于一份数据集，我进行了三个不同模型的搭建：
* 逻辑回归：准确率95.7%
* ANN神经网络：准确率93.6%
* 随机森林：92.8%

结果均能在预测集上表现出很好的效果，实现了中文评论的高准确率情感分析，同时对中文信息处理有了更加深刻的认识。